In [2]:
import random
import re
import spacy
import numpy as np
import json
import string
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config
from iexfinance.stocks import Stock
from wxpy import *

In [3]:
#Create user and bot template
bot_template = "BOT : {0}"
user_template = "USER : {0}"

In [4]:
# Create a trainer that uses this config
trainer = Trainer(config.load("config_spacy.yml"))

# Load the training data
training_data = load_data('training-data.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


In [5]:
#Define basical responses which robot can respond
responses = { 'greet': ['Hello you!', 
                       'Nice to meet you!', 
                       'Hi!', 
                       'How are you!'], 
             'ask_usage': ['I am a stock query robot',
                       'I am a robot that help you search for stock information.', 
                       'I am a robot that can query stock information for you.'],
             'thankyou': ['My pleasure! What else can I do for you?', 
                         'You are welcome! Do you need more information?'
                         ''],
             'bye': ['Bye!', 
                         'Byebye!',
                         'Thank you for using me!:)',
                         'See you!' ],
             'default': ['Sorry, I could not understand you.',
                         'Sorry, I do not understand what you said.',
                         'I don not undersatand you, can you tell me what you want more clearly?']
            }

In [6]:
# Define basical respond() which can recognize user intent and give proper response
def respond(message):
    # Concatenate the user's message to the end of a standard bot respone
    data = interpreter.parse(message)
    intent = data["intent"]['name']
    if intent in responses:
        # Return a random matching response
        bot_message = random.choice(responses[intent])
    else:
        bot_message = chitchat_response(message)
    return bot_message

In [7]:
# Define chitchat_response()
def chitchat_response(message):
    # Call match_rule()
    response, var = match_rule(rules, message)
    # Return none is response is "default"
    if response == "default":
        return random.choice(responses["default"])
    if '{0}' in response:
        # Replace the pronouns of phrase
        phrase = replace_pronouns(message)
        # Calculate the response
        response = response.format(phrase)
    return response

In [8]:
def match_rule(rules, message):
    for pattern, responses in rules.items():
        match = re.search(pattern, message)
        if match is not None:
            response = random.choice(responses)
            var = match.group(1) if '{0}' in response else None
            return response, var
    return "default", None

rules = {'if (.*)': ["Do you really think it's likely that {0}", 'Do you wish that {0}', 'What do you think about {0}', 'Really--if {0}'], 'do you think (.*)': ['if {0}? Absolutely.', 'No chance'], 'I want (.*)': ['What would it mean if you got {0}', 'Why do you want {0}', "What's stopping you from getting {0}"], 'do you remember (.*)': ['Did you think I would forget {0}', "Why haven't you been able to forget {0}", 'What about {0}', 'Yes .. and?']}

def replace_pronouns(message):

    message = message.lower()
    if re.search(r'\bme\b', message):
        return re.sub(r'\bme\b', 'you', message)
    if re.search(r"\b(i)\b", message):
        return re.sub(r"\b(i)\b", 'you', message)
    elif re.search(r'\bmy\b', message):
        return re.sub(r'\bmy\b', 'your', message)
    elif re.search(r'\byour\b', message):
        return re.sub(r'\byour\b', 'my', message)
    elif re.search(r'\byou\b', message):
        return re.sub(r'\byou\b', 'me', message)
    return message

In [9]:
# Define the states
INIT=0
AUTHED=1
STOCK_QUERY=2
ORDERED=3
END=4

# Define the policy rules
policy_rules = {
    (INIT, "stock_search"): (INIT, "you'll have to log in first, what's your phone number?", AUTHED),
    (INIT, "number"): (AUTHED, "perfect, welcome! You can start searching stock information now. Here's the information you can search for: 1. latest price 2. latest volume 3. open price 4. close price 5. real time price", None),
    (AUTHED, "stock_search"): (AUTHED, "Please tell me which stock you want to know.", STOCK_QUERY),
    (AUTHED, "stock_search"): (STOCK_QUERY, "What information do you want to know?", None),    
    (STOCK_QUERY, "stock_search"): (ORDERED, "Here's the information. What else do you need?", None),
    (ORDERED, "stock_search"): (STOCK_QUERY, "What else do you want to know?", None),
    (ORDERED, "stock_search"): (ORDERED, "What else do you want to know?", None),
    (ORDERED, "end"): (END, "Byebye, hope to see you again!", None),
    (STOCK_QUERY, "end"): (END, "Ok bye, hope to see you again!", None),
}

In [10]:
#interpret user's intent and respond information he want to know
def interpret(message):
    msg = message.lower()
    data = interpreter.parse(message)
    intent = data["intent"]['name']
    global param
    if any([d in message for d in string.digits]):
        return 'number' 
    if intent == "bye":
        return 'end'
    if intent == "thankyou":
        response = random.choice(responses["thankyou"])
        print("BOT : {}".format(response))
        return "stock_search"
    if intent == "stock_search":
        entities = data['entities']
        for i in range(len(entities)):
            if data['entities'][i]['entity'] == "company":
                company = data['entities'][i]["value"]
                param = company
        if 'price' in message:
            if ('latest' or 'newest') in message:
                response = respond_latestPrice(param)
                print("BOT : {}".format(response))
            elif 'open' in message:
                response = respond_openPrice(param)
                print("BOT : {}".format(response))
            elif 'close' in message:
                response = respond_closePrice(param)
                print("BOT : {}".format(response))
            else:
                response = respond_realTimePrice(param)
                print("BOT : {}".format(response))
        if 'volume' in message:
            response = respond_latestVolume(param)
            print("BOT : {}".format(response))
        return 'stock_search'  
    return 'none'

In [11]:
def respond_latestPrice(company):
    aapl = Stock(company, token = "pk_05a3d93860d54ae4a2c0f9a01ed362d5")
    info = str(aapl.get_quote()['latestPrice'])
    bot_message =  "Its latest price is " + info +"$"
    return bot_message
    
def respond_latestVolume(company):
    aapl = Stock(company, token = "pk_05a3d93860d54ae4a2c0f9a01ed362d5")
    info = str(aapl.get_quote()['latestVolume'])
    bot_message =  "Its latest volume is " + info +"$"
    return bot_message

def respond_realTimePrice(company):
    aapl = Stock(company, token = "pk_05a3d93860d54ae4a2c0f9a01ed362d5")
    info = str(aapl.get_quote()['iexRealtimePrice'])
    bot_message =  "Its real time price is " + info +"$"
    return bot_message

def respond_openPrice(company):
    aapl = Stock(company, token = "pk_05a3d93860d54ae4a2c0f9a01ed362d5")
    info = str(aapl.get_quote()['open'])
    bot_message =  "Its open price is " + info +"$"
    return bot_message

def respond_closePrice(company):
    aapl = Stock(company, token = "pk_05a3d93860d54ae4a2c0f9a01ed362d5")
    info = str(aapl.get_quote()['close'])
    bot_message =  "Its close price is " + info +"$"
    return bot_message

In [12]:
# Define send_message()
def send_message(state, pending, message):
    response, var = match_rule(rules, message)
    print("USER : {}".format(message))
    if (response == "default"):
        pass
    else:
        response = chitchat_response(message)
        print("BOT : {}".format(response))
    data = interpreter.parse(message)
    intent = data["intent"]['name']
    if (intent == "greet") or (intent == "ask_usage") or (intent == "default"):
        new_state = INIT
        pending = None
        response = respond(message)
        print("BOT : {}".format(response))
    # Calculate the new_state, response, and pending_state
    else:
        new_state, response, pending_state = policy_rules[(state, interpret(message))]
        print("BOT : {}".format(response))
        if pending is not None:
            new_state, response, pending_state =policy_rules[pending]
            print("BOT : {}".format(response))        
        if pending_state is not None:
            pending = (pending_state, interpret(message))
    return new_state, pending, response

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending, response = send_message(state, pending, msg)

In [13]:
# Send the messages
send_messages([
    "hi",
    "what can you do",
    "I want to search some info about stock",
    "555-12345",
    "Show me AAPL's latest price",
    "and its real time price",
    "thank you",
    "I want to know goog's open price",
    "and its real time price",
    "that's it",
    "bye"
])

USER : hi
BOT : Nice to meet you!
USER : what can you do
BOT : I am a robot that help you search for stock information.
USER : I want to search some info about stock
BOT : What's stopping you from getting you want to search some info about stock
BOT : you'll have to log in first, what's your phone number?
USER : 555-12345
BOT : perfect, welcome! You can start searching stock information now. Here's the information you can search for: 1. latest price 2. latest volume 3. open price 4. close price 5. real time price
BOT : What information do you want to know?
USER : Show me AAPL's latest price
BOT : Its latest price is 206.5$
BOT : Here's the information. What else do you need?
BOT : What information do you want to know?
USER : and its real time price
BOT : Its real time price is None$
BOT : Here's the information. What else do you need?
BOT : What information do you want to know?
USER : thank you
BOT : You are welcome! Do you need more information?
BOT : Here's the information. What else

In [17]:
bot = Bot()
my_friend = bot.friends().search('洁', sex=FEMALE, city="上海")

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.


KeyError: 'pass_ticket'

In [15]:
@bot.register(my_friend)
def respond_friend(message):
    state = INIT
    pending = None
    state, pending, response = send_message(state, pending, message)
    message.reply(response)
    return None

NameError: name 'bot' is not defined